In [220]:
# 모든 변수를 포함한 full model
print("full model = const, bp, bmi, age, sex")
R2 = fitted.rsquared
n = fitted.nobs #number of observations
p = fitted.df_model #설명변수의 개수
constant = 'const' in fitted.params.keys() #절편 유무
SSE = sum((fitted.resid)**2) #full model의 잔차제곱합
MSE = SSE / (n-p-1) # fitted.mse_resid (MSE)

adj_R2 = 1- (1-R2)*(n-1)/(n-p-1)
print(" 수정 결정계수: {:.3f}".format(adj_R2))

llf = -n/2*np.log(2*np.pi) - n/2*np.log(SSE / n) - n/2 #로그 가능도 값
aic = -2*llf + 2*(p + constant)
bic = -2*llf + np.log(n)*(p + constant)
print(" AIC: {:.1f}".format(aic))
print(" BIC: {:.1f}".format(bic))

cp = SSE/MSE - n + 2*(p+1)
print(" Cp {:.1f} -> 변수+상수 개수의 합 {}".format(cp, p+constant))

full model = const, bp, bmi, age, sex
 수정 결정계수: 0.376
 AIC: 3293.9
 BIC: 3312.4
 Cp 5.0 -> 변수+상수 개수의 합 5.0


In [222]:
# 일부 변수만 포함한 후보 model
print("후보 model = const, bp, bmi")
fitted2, summary2 = multiR_fitted(X.filter(['bp', 'bmi']), y)
n = fitted2.nobs #number of observations
p = fitted2.df_model #설명변수의 개수
constant = 'const' in fitted2.params.keys() #절편 유무
SSE = sum((fitted2.resid)**2) #full model의 잔차제곱합
MSE = fitted.mse_resid # 비교모델의 MSE가 아닌 full모델의 MSE

adj_R2 = 1- (1-R2)*(n-1)/(n-p-1)
print(" 수정 결정계수: {:.3f}".format(adj_R2))

llf = -n/2*np.log(2*np.pi) - n/2*np.log(SSE / n) - n/2 #로그 가능도 값
aic = -2*llf + 2*(p + constant)
bic = -2*llf + np.log(n)*(p + constant)
print(" AIC: {:.1f}".format(aic))
print(" BIC: {:.1f}".format(bic))

cp = SSE/MSE - n + 2*(p+1)
print(" Cp {:.1f} -> 변수+상수 개수의 합 {}".format(cp, p+constant))

후보 model = const, bp, bmi
 수정 결정계수: 0.380
 AIC: 3293.0
 BIC: 3304.1
 Cp 4.1 -> 변수+상수 개수의 합 3.0


In [208]:
## 단계적 선택법
from pandas import DataFrame
import statsmodels.api as sm
from itertools import combinations
import numpy as np
def stepwise_method(X, y, criterion='AIC'): #'BIC', 'CP', 'adj_R2'
    result = DataFrame()
    feature_combis = [] #변수 조합의 모든 경우의 수
    for i in range(1, len(X.columns)+1):
        feature_combis += list(combinations(X.columns, i))
        feature_combis.reverse() #p개수 내림차순
    feature_combis = [feature_combi for feature_combi in feature_combis if 'const' in feature_combi]
    
    for j, feature_combi in enumerate(feature_combis):
        X_ = X.filter(feature_combi)
        X_ = sm.add_constant(X_) #절편 추가
        model = sm.OLS(y, X_)
        fitted = model.fit() #모델 적합
        n = fitted.nobs #number of observations
        p = fitted.df_model #설명변수의 개수
        if j==0:
            MSE = fitted.mse_resid # full모델의 MSE
        
        #각 기준값 계산
        aic = fitted.aic
        bic = fitted.bic
        cp = sum((fitted.resid)**2)/MSE - (n-2*(p+1))
        adj_R2 = fitted.rsquared_adj
        
        #각 기준값 입력
        result.loc[j, 'feature_combi'] = str(feature_combi)
        result.loc[j, 'AIC'] = aic
        result.loc[j, 'BIC'] = bic
        result.loc[j, 'CP'] = cp
        result.loc[j, 'adj_R2'] = adj_R2
        
        if criterion in ['AIC', 'BIC']: #낮을수록 Best
            result = result.sort_values(by=criterion, ascending = True)
            best = result.iloc[0, 0]
        elif criterion in ['adj_R2']: #높을수록 Best
            result = result.sort_values(by=criterion, ascending = False)
            best = result.iloc[0, 0]
        elif criterion in ['CP']: #CP값이 작고, 변수의 개수와 유사할수록 Best
            best_idx = np.abs(result['CP']-result['feature_combi'].apply(lambda x: x.count(',')+1)).sort_values(ascending=True).index
            result = result.loc[best_idx, :]
            best = result.iloc[0, 0]
        else:
            print("criterion options only cover AIC, BIC, CP, adj_R2.")
    return best, result

In [214]:
# AIC 기준으로 단계적 선택법을 통한 최적의 변수 선택 결과는 다음과 같다.
best, result = stepwise_method(X, y, criterion='AIC')
print("최적의 변수 조합: ", best)
print(f"전체 결과값: \n{result}\n")

for c in ['AIC', 'BIC', 'CP', 'adj_R2']:
    best, result = stepwise_method(X, y, c)
    print(f"기준 {c}에 의한 최적의 변수 조합 {best}")

최적의 변수 조합:  ('const', 'bmi', 'sex', 'bp')
전체 결과값: 
                           feature_combi          AIC          BIC  \
14         ('const', 'bmi', 'sex', 'bp')  3292.208500  3307.023630   
4                 ('const', 'bmi', 'bp')  3293.035334  3304.146682   
0   ('const', 'bmi', 'age', 'sex', 'bp')  3293.858436  3312.377348   
13         ('const', 'bmi', 'age', 'bp')  3294.846640  3309.661769   
6                ('const', 'bmi', 'age')  3317.277536  3328.388883   
12        ('const', 'bmi', 'age', 'sex')  3318.685967  3333.501097   
8                       ('const', 'bmi')  3320.067627  3327.475192   
5                ('const', 'bmi', 'sex')  3321.899726  3333.011073   
11                       ('const', 'bp')  3373.619344  3381.026909   
2                 ('const', 'age', 'bp')  3374.170926  3385.282274   
1                 ('const', 'sex', 'bp')  3374.572580  3385.683927   
15         ('const', 'age', 'sex', 'bp')  3374.858386  3389.673516   
9                       ('const', 'age'